# Aldi vs. Lidl scraping

### Objectives

 - list of postcodes
 - scrape all locations based on postcode list
 - drop duplicates
- get postcodes where 7 characters (only works for 6 character postcodes)
 - drop rows where lat,long == 99,0 <- not yet

In [2]:
import pandas as pd
import requests
import time

In [4]:
# Remove unnecessary data and save as new csv
my_list = pd.read_csv('.ONSPD_NOV_2021_UK/Data/ONSPD_NOV_2021_UK.csv')

In [7]:
# create new dataframe from loaded csv
x = my_list

# format dataframe and save to csv
x = x[['pcd', 'lat', 'long']].reset_index(drop=True)
x[['pc1','pc2']] = x['pcd'].str.split(expand=True)
x.to_csv('postcodes.csv')

# x['pc3'] = x['pcd'].str.replace(' ','')

In [19]:
# drop duplicates from dataframe
y = x.drop_duplicates(subset='pc1')
y['pc1'] = y['pcd'].str[:4]
y['pc2'] = y['pcd'].str[3:8]
y = y.drop_duplicates(subset='pc1').reset_index(drop=True)
y = y[y.lat < 99]
y.to_csv('postcodes_cut.csv')

# z = y['pc3'].squeeze().str.len()
# print(z)
# if length of string in pc3 == 7
# pc2 = pc1.split(4 characters in)
# print(y.iloc[:,0])

In [ ]:
url = 'https://www.aldi.co.uk/api/store-finder/search?q=BL2%203FP&address=Ryeburn%20Dr%2C%20Bolton%20BL2%203FP&latitude=53.6064826&longitude=-2.4095537'
payload = {}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
result = requests.get(url, headers=headers).json()

df = pd.json_normalize(result, record_path='results')
df = df.drop(['distance','openingTimes','isFirstStore','selectedPage'],axis=1)

In [ ]:
data = []
i = 0
# for i in range(len(my_list)):
#   print(my_list.iloc[i, 3], my_list.iloc[i, 4])

for row in my_list.itertuples():
    url = f'https://www.aldi.co.uk/api/store-finder/search?q={getattr(row, "pc1")}%20{getattr(row, "pc2")}&latitude=&longitude='
    print(url)

# for z,y,x,w in my_list[['pc1','pc2','lat','long']].values:
#     url = f'https://www.aldi.co.uk/api/store-finder/search?q={z}%20{y}&latitude={x}&longitude={w}'
#     result = requests.get(url, headers=headers).json()

